In [7]:
import fairseq
import torch
import os
import glob
from fairseq.models.wav2vec import Wav2VecModel
from fairseq.models.roberta import RobertaModel
import tqdm
import librosa
import math

In [8]:
roberta_model_path = '/dfs/user/prabhat8/CS224S/models/roberta.large'
data_path = '/dfs/user/prabhat8/CS224S/gad'

In [ ]:
roberta = RobertaModel.from_pretrained(roberta_model_path, checkpoint_file='model.pt')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

In [20]:
import pickle
with torch.no_grad():
    for transcript_file in tqdm.tqdm(glob.glob(f"{data_path}/transcripts/*.txt")):
        fname_feat = os.path.basename(transcript_file)[:-4] + "_feat.p"
        fname_tok = os.path.basename(transcript_file)[:-4] + "_tokens.p"
        with open(transcript_file) as f:
            text = f.readlines()
            if type(text) == list:
                if text:
                    text = text[0]
                else:
                    text = ""
        tokens = roberta.encode(text)[:512]
        pickle.dump(tokens, open(os.path.join(data_path, 'transcripts_tokens', fname_tok), 'wb') )
        last_layer_features = roberta.extract_features(tokens)
#         print(last_layer_features.shape)
        pickle.dump(last_layer_features, open(os.path.join(data_path, 'transcripts_features', fname_feat), 'wb') )
        

100%|██████████| 2257/2257 [14:37<00:00,  2.57it/s]
